In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import MetaTrader5 as mt5
import pytz
from datetime import datetime, timedelta
import time


In [2]:
# establish connection to the MetaTrader 5 terminal
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()

In [3]:
# connect to the trade account without specifying a password and a server
account=511899
authorized=mt5.login(account, password= "Your_password", server="Pepperstone-MT5-Live01", path= "C:\Program Files\Pepperstone MetaTrader 5\terminal64.exe")  

# the terminal database password is applied if connection data is set to be remembered
if authorized:
    account_info_dict =mt5.account_info()._asdict()
    for prop in account_info_dict:
        print(" {}={}".format(prop, account_info_dict[prop]))
else:
    print("failed to connect at account #{}, error code: {}".format(account, mt5.last_error()))
    


 login=511899
 trade_mode=2
 leverage=200
 limit_orders=500
 margin_so_mode=0
 trade_allowed=True
 trade_expert=True
 margin_mode=2
 currency_digits=2
 fifo_close=False
 balance=151.51
 credit=0.0
 profit=0.0
 equity=151.51
 margin=0.0
 margin_free=151.51
 margin_level=0.0
 margin_so_call=90.0
 margin_so_so=50.0
 margin_initial=0.0
 margin_maintenance=0.0
 assets=0.0
 liabilities=0.0
 commission_blocked=0.0
 name=david poblano
 server=Pepperstone-MT5-Live01
 currency=USD
 company=Pepperstone Group Limited


In [4]:
# Definir el símbolo y el timeframe
symbol = "EURUSD"
timeframe = mt5.TIMEFRAME_M5

# Establecer la zona horaria a UTC
timezone = pytz.timezone("Etc/UTC")

# Obtener la fecha y hora actual en la zona horaria UTC
now_utc = datetime.now(tz=timezone)

# Calcular la fecha de "ayer"
yesterday_utc = now_utc - timedelta(days=1)

# Crear objetos 'datetime' en la zona horaria UTC para ayer y hoy
utc_from = yesterday_utc
utc_to = now_utc

# Imprimir información antes de obtener los datos
print(f"Solicitando datos históricos para {symbol} - Timeframe: {timeframe}")
print(f"Desde: {utc_from} - Hasta: {utc_to}")

Solicitando datos históricos para EURUSD - Timeframe: 5
Desde: 2023-08-31 02:51:24.170767+00:00 - Hasta: 2023-09-01 02:51:24.170767+00:00


In [5]:
# Obtener datos históricos usando copy_rates_range
rates = mt5.copy_rates_range('EURUSD', mt5.TIMEFRAME_M5, utc_from, utc_to)

# Imprimir información después de obtener los datos
print(f"Total de barras obtenidas: {len(rates)}")
print("Primeras 10 barras obtenidas:")
for i, rate in enumerate(rates[:20], start=1):
    print(f"Barra {i}: {rate}")

Total de barras obtenidas: 288
Primeras 10 barras obtenidas:
Barra 1: (1693450500, 1.09291, 1.09296, 1.09289, 1.09295, 40, 0, 0)
Barra 2: (1693450800, 1.09295, 1.09319, 1.0928, 1.09316, 147, 0, 0)
Barra 3: (1693451100, 1.09315, 1.09315, 1.09286, 1.09306, 160, 0, 0)
Barra 4: (1693451400, 1.09304, 1.0932, 1.09286, 1.09289, 88, 0, 0)
Barra 5: (1693451700, 1.09289, 1.09315, 1.09289, 1.09304, 78, 0, 0)
Barra 6: (1693452000, 1.09304, 1.0932, 1.09298, 1.0931, 113, 0, 0)
Barra 7: (1693452300, 1.0931, 1.09312, 1.09297, 1.09307, 88, 0, 0)
Barra 8: (1693452600, 1.09307, 1.09321, 1.09299, 1.09315, 91, 0, 0)
Barra 9: (1693452900, 1.09315, 1.09316, 1.09298, 1.09301, 57, 0, 0)
Barra 10: (1693453200, 1.09305, 1.0931, 1.09298, 1.09303, 45, 0, 0)
Barra 11: (1693453500, 1.09303, 1.09304, 1.09292, 1.09302, 104, 0, 0)
Barra 12: (1693453800, 1.09302, 1.0934, 1.09291, 1.0934, 220, 0, 0)
Barra 13: (1693454100, 1.09326, 1.09338, 1.09322, 1.09331, 164, 0, 0)
Barra 14: (1693454400, 1.09331, 1.09353, 1.09323, 1.0

In [6]:
# Cargar el modelo LSTM previamente entrenado
model = tf.keras.models.load_model('model_checkpoint.h5')

In [7]:
# Extraer los precios de cierre de los datos históricos
close_prices = np.array([bar[4] for bar in rates])

# Normalizar los precios de cierre
scaler = MinMaxScaler()
scaler.fit(close_prices.reshape(-1, 1))
scaled_prices = scaler.transform(close_prices.reshape(-1, 1))

print(close_prices)


[1.09295 1.09316 1.09306 1.09289 1.09304 1.0931  1.09307 1.09315 1.09301
 1.09303 1.09302 1.0934  1.09331 1.09344 1.09341 1.09345 1.09367 1.09351
 1.09346 1.09389 1.09367 1.09371 1.09355 1.09321 1.09289 1.09269 1.09239
 1.09247 1.0927  1.09276 1.09255 1.0925  1.09236 1.09237 1.09227 1.09222
 1.09218 1.09224 1.09222 1.09213 1.09224 1.09214 1.09225 1.09214 1.0922
 1.09203 1.09211 1.09209 1.09204 1.09214 1.09218 1.09224 1.09217 1.09202
 1.09204 1.09205 1.09201 1.09192 1.09201 1.09195 1.09187 1.09178 1.09169
 1.09179 1.0918  1.09182 1.09173 1.09189 1.09186 1.09173 1.0919  1.09191
 1.0918  1.09137 1.09138 1.09154 1.09145 1.09113 1.09135 1.09135 1.09148
 1.09165 1.09158 1.09092 1.09072 1.0903  1.08969 1.08942 1.08922 1.08942
 1.08972 1.09014 1.09032 1.08997 1.08997 1.08937 1.08927 1.08918 1.08858
 1.08893 1.08893 1.08877 1.08865 1.08856 1.08884 1.08871 1.08843 1.08813
 1.08779 1.08654 1.08709 1.08711 1.08716 1.08739 1.08734 1.08725 1.08704
 1.08735 1.08734 1.08744 1.08742 1.08733 1.08707 1.0

In [8]:
# Preparar los datos y generar predicciones
sequence_length = 15  # Ajusta la longitud de la secuencia a 15
sequences = []
for i in range(len(scaled_prices) - sequence_length):
    sequences.append(scaled_prices[i:i+sequence_length])

X_input = np.array(sequences)

# Generar predicciones usando el modelo LSTM
predictions = model.predict(X_input)

# Desnormalizar las predicciones
predicted_prices = scaler.inverse_transform(predictions)

9/9 [==============================] - 0s 1ms/step


In [9]:
# Implementar una estrategia de trading simple
lot_size = 0.02  # Tamaño del lote
comment = "Orden de compra"
interval_between_orders = 300

# Definir las constantes de tipo de órdenes
ORDER_TYPE_BUY = mt5.ORDER_TYPE_BUY
ORDER_TYPE_SELL = mt5.ORDER_TYPE_SELL


In [ ]:
# Ejecutar la estrategia en un bucle infinito
while True:
    for i in range(len(predicted_prices)):
        if i == 0:
            continue
        predicted_price = float(predicted_prices[i][0])  # Extraer un valor escalar del array
        previous_price = float(predicted_prices[i-1][0])  # Agregar esta línea para tener el precio anterior

        # Calcular el valor en pips para TP y SL
        tp_pips = 50
        sl_pips = 125

        # Definir el tipo de orden basado en la predicción
        if predicted_price > previous_price:
            order_type = ORDER_TYPE_BUY
            price = mt5.symbol_info_tick(symbol).ask
            take_profit_price = price + tp_pips * mt5.symbol_info(symbol).point
            stop_loss_price = price - sl_pips * mt5.symbol_info(symbol).point
        else:
            order_type = ORDER_TYPE_SELL
            price = mt5.symbol_info_tick(symbol).bid
            take_profit_price = price - tp_pips * mt5.symbol_info(symbol).point
            stop_loss_price = price + sl_pips * mt5.symbol_info(symbol).point

        # Preparar la estructura de la solicitud de orden
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": lot_size,
            "type": order_type,
            "price": price,
            "tp": take_profit_price,
            "sl": stop_loss_price,
            "deviation": 20,
            "magic": 234000,
            "comment": comment,
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }

        # Enviar la solicitud de trading
        result = mt5.order_send(request)

        # Verificar el resultado de la ejecución
        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print("Orden fallida, retcode={}".format(result.retcode))
            # Puedes agregar más detalles sobre el resultado aquí
        else:
            print("Orden exitosa")

        # Esperar un tiempo antes de la siguiente iteración
        time.sleep(interval_between_orders)

Orden exitosa
Orden exitosa
Orden exitosa
Orden exitosa
